# Iteration 6: Summary

This was my first ssuccessful iteration.  Using the model from the previous iteration, I trained for 30 epochs, with a batch size of 64, and using the Adam optimizer with a learn rate of 0.01.

I was not able to get results from the test dataset, but the training ended with a validation accuracy of 0.68 and a validation loss of 0.72.

import shutil
import os

source = './food-101/test/ramen'
dest = './food-101/valid/ramen'

files = os.listdir(source)

i = 0
for f in files:
    if i < 175:
        file= os.path.join(source, f)
        shutil.move(file,dest)
        print("adding img ", i)
    i+=1

# Preprocessing and Loading Data

For loading the data, I first used the train/test split that was provided with our dataset.  This gave us 750 images in train and 250 in test PER CLASS.  Then, I split the test set into test & validation.  I moved 175 images to the validation set, thus we are left with 75 images in test.

For image preprocessing, I first resize each image to our IMG_SIZE constant, currently set to 224 pixels.  I then recolor the image to RGB.  I also assign the labels to the images and shuffle the dataset.


After loading the data, the shape of X (features) will be (-1 {this means any number of features}, IMG_SIZE, IMG_SIZE, 3 {number of channels - RGB}), and the shape of y (labels) will be (-1, 1).

In [1]:
# importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt

import os
from os import listdir
from os.path import isfile, join

import cv2
import random
import datetime
import time
import shutil
import stat

import collections
from collections import defaultdict

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical, plot_model

from keras.models import Model

Using TensorFlow backend.


In [2]:
# constants
IMG_SIZE = 224
NUM_CLASSES = 5
noodle_classes = ['pad_thai','pho','ramen','spaghetti_bolognese','spaghetti_carbonara']

# these are the main variables we tested and documented
EPOCHS = 30
BATCH_SZ = 64
LEARN_RATE = 0.01

In [3]:
# function to create the datasets (train, test, and valid)
# preprocesses the images (resize/recolor), assigns labels, and shuffles the dataset
root_dir = 'food-101/images/'

def create_data(root, _name):
    data_set = []

    for noodle_class in noodle_classes:
        path = os.path.join(root, noodle_class)
        label = noodle_classes.index(noodle_class) # assign an integer label to the image based on our noodle_classes array
        print(noodle_class,label)

        for img in os.listdir(path): # iterates through each image in the noodle folder
                if img.startswith('.'):
                    continue
                # each image is a 2D array of RGB value
                try:
                    img_arr = cv2.imread(os.path.join(path,img)) 
                    img_to_rgb = img_arr[:,:,::-1] # recolor
                    new_img = cv2.resize(img_to_rgb,(IMG_SIZE,IMG_SIZE)) #resize
                    data_set.append([new_img,label]) # store image and label together in dataset so we can shuffle without images getting mislabeled
                except Exception as e: # catch bad images
                    print("create exception: ",e)
        
    # randomize
    random.shuffle(data_set)
    
    X = []
    y = []

    for features, label in data_set:
        X.append(features) # 2D array of RGB values representing features
        y.append(label) # integer representing class/label

    # reshape X and y
    X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
    y = np.array(y).reshape(-1, 1)
    print('X', X.shape)
    print('y', y.shape)

    return X,y

In [4]:
# load data in train, test, and valid

X_train, y_train = create_data('food-101/train', "train")
X_test, y_test = create_data('food-101/test', "test")
X_valid, y_valid = create_data('food-101/valid', "valid")

pad_thai 0
pho 1
ramen 2
spaghetti_bolognese 3
spaghetti_carbonara 4
X (3750, 224, 224, 3)
y (3750, 1)
pad_thai 0
pho 1
ramen 2
spaghetti_bolognese 3
spaghetti_carbonara 4
X (375, 224, 224, 3)
y (375, 1)
pad_thai 0
pho 1
ramen 2
spaghetti_bolognese 3
spaghetti_carbonara 4
X (875, 224, 224, 3)
y (875, 1)


# Data Normalization and Augmentation
Here I am normalizing the data to scale our input training vectors. This will help improve accuracy and increase training speed.

Currently, I am attempting to implement image augmentation to improve accuracy as well.

In [5]:
# normalize data
# X_train = X_train/255.0
# X_test = X_test/255.0

#print('X train before normalize',X_train[1])
print('y train before normal',y_train)
# input('wait1')

# L2-normalizes the given array, i.e., it makes the sum of squares of each element of the array to be equal to one
X_train = tf.keras.utils.normalize(X_train, axis=1)
y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)

#print('X train after normal',X_train[1])
print('y train after normal',y_train)

X_test = tf.keras.utils.normalize(X_test, axis=1)
y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

X_valid = tf.keras.utils.normalize(X_valid, axis=1)
y_valid = tf.keras.utils.to_categorical(y_valid, NUM_CLASSES)

print(y_train.shape)

y train before normal [[0]
 [3]
 [3]
 ...
 [2]
 [2]
 [0]]
y train after normal [[1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]
(3750, 5)


y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
print('y train after normal',y_train)

y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

y_valid = tf.keras.utils.to_categorical(y_valid, NUM_CLASSES)

In [6]:
from keras.applications.vgg16 import preprocess_input

# image augmentation - for better performance
datagen = ImageDataGenerator(
    rotation_range = 90,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,
    brightness_range=[0.2,1.0],
    zoom_range=[0.5,1.0]
    #preprocessing_function=preprocess_input
)

datagener = datagen.flow(X_train, y_train, batch_size = BATCH_SZ, shuffle=True)
datagen.fit(X_train)

# image augmentation - for better performance
vdatagen = ImageDataGenerator(
    rotation_range = 90,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,
    brightness_range=[0.2,1.0],
    zoom_range=[0.5,1.0]
    #preprocessing_function=preprocess_input
)

vdatagener = vdatagen.flow(X_valid, y_valid, batch_size = BATCH_SZ, shuffle=True)
vdatagen.fit(X_valid)


# VGGNet

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization, LeakyReLU
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.utils import plot_model

# creating the actual model using the VGGNet16 architecture

input_shape = (IMG_SIZE, IMG_SIZE, 3) # 224x224x3 RGB image

#Instantiate an empty model
model = Sequential([
Conv2D(64, (3, 3), input_shape=input_shape, kernel_initializer='glorot_normal', bias_initializer='zeros', padding='same'), # first two convolutional layers
LeakyReLU(alpha=0.01),
Conv2D(64, (3, 3), padding='same'),
LeakyReLU(alpha=0.01),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(128, (3, 3), padding='same'), 
LeakyReLU(alpha=0.01),
Conv2D(128, (3, 3), padding='same',), 
LeakyReLU(alpha=0.01),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(256, (3, 3), padding='same',),
LeakyReLU(alpha=0.01),
Conv2D(256, (3, 3), padding='same',),
LeakyReLU(alpha=0.01),
Conv2D(256, (3, 3), padding='same',),
LeakyReLU(alpha=0.01),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)), 
Flatten(), 
Dense(1024), # fully connected layers
LeakyReLU(alpha=0.01),
Dense(1024),
LeakyReLU(alpha=0.01),
Dense(5, activation='softmax') # softmax output layer, 5 possible values/classes
])

model.summary()

#plot_model(model, show_shapes=True, to_file='vgg_block.png')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 224, 224, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 224, 224, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 112, 112, 128)    

# Training

In [8]:
# compiling model and training

opt = optimizers.SGD(lr=LEARN_RATE, momentum=.9) # momentum help accelerate gradient vectors in the right directions

model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])
'''


# Compile the model
model.compile(loss="categorical_crossentropy", optimizer='SGD', metrics=['accuracy'])
'''
print(X_train.shape)
print(y_train.shape)

'''
# fitting model without image augmentation
model.fit(X_train, 
          y_train, 
          batch_size=BATCH_SZ, 
          epochs=EPOCHS,     
          validation_data=(X_valid, y_valid))
'''

# fitting model with image augmentation
model.fit_generator(datagener,
                    steps_per_epoch=X_train.shape[0] // BATCH_SZ,
                    epochs=EPOCHS,
                    validation_data=vdatagener,
                    shuffle=True)

'''
# fitting model with image standardization using ImageDataGenerator
model.fit_generator(train_iterator,
                    steps_per_epoch=len(train_iterator),
                    epochs=EPOCHS)
'''

(3750, 224, 224, 3)
(3750, 5)
Epoch 1/30
58/58 [==============================] - 1178s 20s/step - loss: 20.3834 - accuracy: 0.2018 - val_loss: 1.5462 - val_accuracy: 0.2389
Epoch 2/30
58/58 [==============================] - 1139s 20s/step - loss: 1.5138 - accuracy: 0.3180 - val_loss: 1.4924 - val_accuracy: 0.4057
Epoch 3/30
58/58 [==============================] - 1157s 20s/step - loss: 1.4618 - accuracy: 0.3635 - val_loss: 1.4698 - val_accuracy: 0.3703
Epoch 4/30
58/58 [==============================] - 1154s 20s/step - loss: 1.4261 - accuracy: 0.3869 - val_loss: 1.2519 - val_accuracy: 0.4194
Epoch 5/30
58/58 [==============================] - 1158s 20s/step - loss: 1.3473 - accuracy: 0.4308 - val_loss: 1.4079 - val_accuracy: 0.4789
Epoch 6/30
58/58 [==============================] - 1151s 20s/step - loss: 1.3059 - accuracy: 0.4440 - val_loss: 1.1791 - val_accuracy: 0.4891
Epoch 7/30
58/58 [==============================] - 1163s 20s/step - loss: 1.2787 - accuracy: 0.4674 - val_loss

'\n# fitting model with image standardization using ImageDataGenerator\nmodel.fit_generator(train_iterator,\n                    steps_per_epoch=len(train_iterator),\n                    epochs=EPOCHS)\n'

# Evaluation

In [9]:
# evaluate model on test data
scores = model.evaluate(X_test, y_test, verbose=1)
# verbose indiciator to display training progress info
# 1 (true) (default) | 0 (false)
#scores = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

375/375 [==============================] - 30s 81ms/step
Test loss: 2.1899173425038656
Test accuracy: 0.20000000298023224


In [ ]:
# image augmentation - for better performance
tdatagen = ImageDataGenerator(
    rotation_range = 90,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,
    brightness_range=[0.2,1.0],
    zoom_range=[0.5,1.0]
    #preprocessing_function=preprocess_input
)

tdatagener = vdatagen.flow(X_test, y_test, batch_size = BATCH_SZ)
tdatagen.fit(X_test)

scores = model.evaluate_generator(tdatagener, verbose=1)
print('Test Loss: ', scores[0])
print('Test Accuracy: ', scores[1])